# 파일에 대해 적용_가사ver(전처리 후)

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import re

# 의미가 부족한 단어 리스트
meaningless_words = ['담은', '표현', '대한']

# 영어 단어를 제거하는 함수
def remove_english_words(sentence):
    return re.sub(r'[a-zA-Z]', '', sentence)

# 조사 및 특수 문자를 제거하고 중복된 문장을 제거하는 함수
def remove_duplicates(sentences):
    cleaned_sentences = {}
    for sentence in sentences:
        # 영어 단어 제거
        sentence = remove_english_words(sentence)
        # 조사 및 특수 문자 제거
        cleaned_sentence = re.sub(r'[^\w\s]', '', sentence).lower()
        # 조사 제거
        cleaned_sentence = re.sub(r'(으|이|가|을|를|은|는|의|에|로|으로|와|과|도|만|나|고|며|부터)$', '', cleaned_sentence)
        if cleaned_sentence not in cleaned_sentences:
            cleaned_sentences[cleaned_sentence] = sentence
    return list(cleaned_sentences.values())

# 긴 문장 분할
def split_long_sentences(sentences, max_length=20):
    split_sentences = []
    for sentence in sentences:
        words = sentence.split()
        split_sentences.extend([' '.join(words[i:i+max_length]) for i in range(0, len(words), max_length)])
    return split_sentences

# CSV 파일 로드
df = pd.read_csv('combined_dataset6.csv')

# 분류별 주요 문장을 저장할 딕셔너리
category_sentences = defaultdict(list)

# TextRank 알고리즘 적용
def apply_textrank(text, top_n=10):
    sentences = text.strip().split('\n')
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    cosine_sim_matrix = cosine_similarity(X, X)
    nx_graph = nx.from_numpy_array(cosine_sim_matrix)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    return ranked_sentences[:top_n]

# 각 카테고리별 중요한 문장 추출 및 키워드 선별
final_keywords = {}
for _, row in df.iterrows():
    category = row['소분류']
    lyrics = row['가사']
    important_sentences = apply_textrank(lyrics)
    category_sentences[category].extend([s for _, s in important_sentences])

# 중요한 단어 추출 및 의미가 부족한 단어 및 영어 단어 제거
for category, sentences in category_sentences.items():
    combined_text = " ".join(remove_duplicates(split_long_sentences(sentences)))
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([combined_text])
    feature_array = np.array(vectorizer.get_feature_names_out())
    tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]
    top_keywords = feature_array[tfidf_sorting][:20]  # 상위 20개 키워드 추출
    meaningful_keywords = [word for word in top_keywords if word not in meaningless_words]
    final_keywords[category] = list(dict.fromkeys(meaningful_keywords))[:10]  # 중복 제거 후 상위 10개 의미있는 키워드 선택

# 결과 출력
for category, keywords in final_keywords.items():
    print(f"Category: {category}")
    print("Top 10 Meaningful Keywords:", keywords)
    print("\n")


Category: 가족
Top 10 Meaningful Keywords: ['엄마', '내가', '나의', '이제', '사람', '아빠', '우리', '울어', '나는', '아버지']


Category: 가족과의 관계
Top 10 Meaningful Keywords: ['엄마', '내가', '아리랑', '너무', '그랬는지', '사람', '이제', '이렇게', '말씀이', '우리']


Category: 감정
Top 10 Meaningful Keywords: ['다시', '혼자', '네가', '나를', '챔피언', '그리워', '니가', '우울하다', '내가', '우울해']


Category: 개인
Top 10 Meaningful Keywords: ['너와', '머리', '무릎', '어깨', '않아', '더는', '없소', '두고', '오직', '누구']


Category: 꿈
Top 10 Meaningful Keywords: ['날아올라', '너를', '나를', '높이', '길을', '가게', '내가', '꿈을', '꿈이', '멀리']


Category: 사랑
Top 10 Meaningful Keywords: ['내가', '너의', '그대', '너무', '많이', '너를', '지금', '좋은', '안녕', '나를']


Category: 사랑의 다양한 양상
Top 10 Meaningful Keywords: ['사랑', '다시', '사람', '나의', '이젠', '나를', '너무', '내가', '그대', '사랑해']


Category: 사회 비판
Top 10 Meaningful Keywords: ['혼자', '나는', '나의', '사랑을', '있어', '함께', '하늘로', '길에', '있는', '그게']


Category: 성장
Top 10 Meaningful Keywords: ['내가', '거야', '나의', '너의', '함께', '나를', '다시', '이태원', '괜찮아', '있어']


Category: 우정
Top 10 Meaningful 

# 예시 코드

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize

# 제공된 가사
lyrics = """
아무리 기다려도 난 못가
바보처럼 울고 있는 너의 곁에
상처만 주는 나를 왜 모르고
기다리니 떠나가란 말야
보고 싶다 보고 싶다
이런 내가 미워질만큼
울고 싶다 네게 무릎꿇고
모두 없던 일이 될 수 있다면
미칠듯 사랑했던 기억이
추억들이 너를 찾고 있지만
더 이상 사랑이란 변명에
너를 가둘수 없어
이러면 안되지만
죽을만큼 보고 싶다
보고 싶다 보고 싶다
이런 내가 미워질만큼
믿고 싶다 옳은 길이라고
너를 위해 떠나야만 한다고
미칠듯 사랑했던 기억이
추억들이 너를 찾고 있지만
더 이상 사랑이란 변명에
너를 가둘수 없어
이러면 안되지만
죽을만큼 보고 싶다
죽을만큼 잊고 싶다.
"""

# 문장 분리
sentences = sent_tokenize(lyrics)

# 분리된 문장들 확인
sentences[:5]  # 첫 5개 문장만 출력하여 확인

['\n아무리 기다려도 난 못가\n바보처럼 울고 있는 너의 곁에\n상처만 주는 나를 왜 모르고\n기다리니 떠나가란 말야\n보고 싶다 보고 싶다\n이런 내가 미워질만큼\n울고 싶다 네게 무릎꿇고\n모두 없던 일이 될 수 있다면\n미칠듯 사랑했던 기억이\n추억들이 너를 찾고 있지만\n더 이상 사랑이란 변명에\n너를 가둘수 없어\n이러면 안되지만\n죽을만큼 보고 싶다\n보고 싶다 보고 싶다\n이런 내가 미워질만큼\n믿고 싶다 옳은 길이라고\n너를 위해 떠나야만 한다고\n미칠듯 사랑했던 기억이\n추억들이 너를 찾고 있지만\n더 이상 사랑이란 변명에\n너를 가둘수 없어\n이러면 안되지만\n죽을만큼 보고 싶다\n죽을만큼 잊고 싶다.']

In [ ]:
# 한국어 가사이므로 각 줄을 별도의 문장으로 취급하여 분리
sentences = lyrics.strip().split('\n')

# 분리된 문장들 확인 (첫 5개 문장 출력)
sentences[:5]


['아무리 기다려도 난 못가',
 '바보처럼 울고 있는 너의 곁에',
 '상처만 주는 나를 왜 모르고',
 '기다리니 떠나가란 말야',
 '보고 싶다 보고 싶다']

In [ ]:
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 통해 문장 간 유사도 계산
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)

# 코사인 유사도 행렬 계산
cosine_sim_matrix = cosine_similarity(X, X)

# 문장 간 유사도를 기반으로 그래프 생성
nx_graph = nx.from_numpy_array(cosine_sim_matrix)
scores = nx.pagerank(nx_graph)

# 가장 중요도가 높은 문장들을 추출 (상위 3개)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
summary = "\n".join([s for _, s in ranked_sentences[:3]])

summary


'죽을만큼 보고 싶다\n죽을만큼 보고 싶다\n보고 싶다 보고 싶다'

In [ ]:
# 요약문을 하나의 문장으로 표현하기 위해 가장 높은 점수를 받은 문장 선택
single_sentence_summary = ranked_sentences[0][1]
single_sentence_summary

'죽을만큼 보고 싶다'

In [ ]:
from collections import OrderedDict
import numpy as np
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_keywords(text):
    # 문장 분리
    sentences = text.strip().split('\n')

    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(sentences)

    # 단어 리스트
    words = vectorizer.get_feature_names_out()

    # 코사인 유사도 행렬 계산
    cosine_sim_matrix = cosine_similarity(X.T, X.T)

    # 단어 간 유사도를 기반으로 그래프 생성
    nx_graph = nx.from_numpy_array(cosine_sim_matrix)
    scores = nx.pagerank(nx_graph)

    # 가장 중요한 단어들 추출 (상위 5개)
    ranked_words = sorted(((scores[i], w) for i, w in enumerate(words)), reverse=True)
    keywords = [w for _, w in ranked_words[:5]]

    return keywords

# 예시 가사
example_lyrics = """
아무리 기다려도 난 못가
바보처럼 울고 있는 너의 곁에
상처만 주는 나를 왜 모르고
기다리니 떠나가란 말야
보고 싶다 보고 싶다
이런 내가 미워질만큼
울고 싶다 네게 무릎꿇고
모두 없던 일이 될 수 있다면
미칠듯 사랑했던 기억이
추억들이 너를 찾고 있지만
더 이상 사랑이란 변명에
너를 가둘수 없어
이러면 안되지만
죽을만큼 보고 싶다
보고 싶다 보고 싶다
이런 내가 미워질만큼
믿고 싶다 옳은 길이라고
너를 위해 떠나야만 한다고
미칠듯 사랑했던 기억이
추억들이 너를 찾고 있지만
더 이상 사랑이란 변명에
너를 가둘수 없어
이러면 안되지만
죽을만큼 보고 싶다
죽을만큼 잊고 싶다.
"""

# 키워드 추출
extracted_keywords = extract_keywords(example_lyrics)
extracted_keywords


['너를', '싶다', '울고', '있는', '바보처럼']

# 파일에 대해 적용_가사ver

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import itertools
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# CSV 파일 로드
df = pd.read_csv('combined_dataset6.csv')
df

,노래제목,가수,가사,요약,소분류,대분류
0,할아버지,여유와 설빈,할아버지는 살아계실 때 지금 내가 앉은 소파에 앉아 신호가 잘 잡히지 않는 라디오를...,"할아버지와의 추억과 그분의 삶을 회상하며, 할아버지에 대한 깊은 그리움과 사랑을 표...",가족,관계
1,아빠가 딸에게 (To.Jia),변진섭,햇살을 닮아 맘이 따뜻한 너에게 무슨 말을 할까 구름을 닮아 선한 눈빛을 가진 너에...,"부모의 따뜻한 마음과 바람을 담은 노래, 자녀에게 전하는 사랑과 삶의 교훈을 표현하...",가족,관계
2,딸에게 보내는 노래 (Vocal 성시경),토이,세상 모두 멈춘 것 같은 밤 방 안 가득 별빛 쏟아져 내려 지친 하루 피곤한 모습의...,어린 자녀와의 소중한 순간들을 회상하며 부모의 사랑과 희망을 담은 노래.,가족,관계
3,아빠와 함께 왈츠를 (Duet With 홍수연),김현철,아빠 나도 누군가를 사랑하게 될까 이 다음에 언젠가 내가 그런 때가 되면 아빤 내게...,아버지와 딸의 깊은 유대감과 미래의 사랑에 대한 소망을 표현한 노래,가족,관계
4,엄마와 딸,인순이,예쁘게 머리 땋아줘서 있는 짜증 없는 짜증 받아줘서 고마워 고마워 고마워 엄마 안기...,"엄마와 딸 사이의 깊은 사랑과 미안함, 그리고 서로에 대한 감사의 마음을 표현한 노래.",가족,관계
...,...,...,...,...,...,...
513,나에게로 떠나는 여행,NaN,저 푸른바다 끝까지\n말을 달리면\n소금같은 별이 떠있고\n사막엔 낙타만이 가는길\...,자유롭게 떠나고자 하는 강한 의지를 담은 노래,환경,일상
514,제주도의 푸른 밤,태연,떠나요 둘이서\n모든 걸 훌훌 버리고\n제주도 푸른 밤 그 별 아래\n이제는 더이상...,도시의 번잡함과 지쳐한 일상에서 벗어나 제주도의 평화로운 푸른 밤에 둘이서 행복한 ...,환경,일상
515,새들처럼,ft아일랜드,열린 공간 속을 가르며\n달려가는 자동차와\n석양에 비추인 사람들\nUh 도로 위를...,도심의 분주함에서 벗어나 파란 하늘 아래로 날아가고 싶어하는 자유로운 마음을 담은 노래,환경,일상
516,음파음파,레드벨벳,Umpah umpah umpah umpah\nUmpah umpah umpah ump...,해변에서의 여름 휴가와 자연을 즐기며 미소지음,환경,일상


In [ ]:
def apply_textrank(text, top_n=10):
    # 문장 분리
    sentences = text.strip().split('\n')
    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    # 코사인 유사도 행렬 계산
    cosine_sim_matrix = cosine_similarity(X, X)
    # 그래프 생성 및 PageRank 적용
    nx_graph = nx.from_numpy_array(cosine_sim_matrix)
    scores = nx.pagerank(nx_graph)
    # 점수가 높은 문장 추출
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    return ranked_sentences[:top_n]  # 상위 n개 문장과 점수 반환

import re

# 특수 문자를 제거하고 중복된 문장을 제거하는 함수
def remove_duplicates(sentences):
    cleaned_sentences = {}
    for sentence in sentences:
        # 특수 문자 제거 및 소문자 변환
        cleaned_sentence = re.sub(r'[^\w\s]', '', sentence).lower()
        if cleaned_sentence not in cleaned_sentences:
            cleaned_sentences[cleaned_sentence] = sentence
    return list(cleaned_sentences.values())

# 긴 문장 분할
def split_long_sentences(sentences, max_length=20):
    split_sentences = []
    for sentence in sentences:
        words = sentence.split()
        # 문장을 최대 길이로 나누어 리스트에 추가
        split_sentences.extend([' '.join(words[i:i+max_length]) for i in range(0, len(words), max_length)])
    return split_sentences

In [ ]:
# 분류별 주요 문장을 저장할 딕셔너리
category_sentences = defaultdict(list)

for _, row in df.iterrows():
    category = row['소분류']
    lyrics = row['가사']

    # TextRank 적용
    important_sentences = apply_textrank(lyrics)

    # 결과 저장
    category_sentences[category].extend([s for _, s in important_sentences])

In [ ]:
# 각 카테고리별 중요한 문장들에 대해 TextRank 재적용
final_ranked_sentences = {}
for category, sentences in category_sentences.items():
    combined_text = "\n".join(remove_duplicates(split_long_sentences(sentences)))
    final_ranked_sentences[category] = apply_textrank(combined_text, top_n=10)

In [ ]:
# 상위 10개 문장과 중요도 점수 출력
for category, sentences_scores in final_ranked_sentences.items():
    print(f"Category: {category}")
    print("Top 10 Important Sentences with Scores:")
    for score, sentence in sentences_scores:
        print(f"Score: {score:.4f}, Sentence: {sentence}")
    print("\n")

Category: 가족
Top 10 Important Sentences with Scores:
Score: 0.0062, Sentence: 많이 사랑해요 엄마 엄마 우리 엄마 세상 가장 큰 선물 내게는 가장 큰 보물 엄마 엄마 우리 엄마 사랑해요 우리
Score: 0.0060, Sentence: 엄마 엄마 우리 엄마 세상 가장 큰 선물 내게는 가장 큰 보물 엄마 엄마 우리 엄마 사랑해요 우리 엄마 고맙고
Score: 0.0054, Sentence: 하는 당신께 더 무엇을 드려야 할지 엄마 나의 어머니 왜 이렇게 눈물이 나죠 가장 소중한 누구보다 아름다운 당신은 나의 나의
Score: 0.0053, Sentence: 엄마 나의 어머니 왜 이렇게 눈물이 나죠 가장 소중한 누구보다 아름다운 당신은 나의 나의 어머니 당신은 나의 나의 어머니
Score: 0.0049, Sentence: 나의 전부 나의 사랑 너만 있으면 살 수 있어 우리 예쁜 딸 세상에 가장 소중한 특별한 사람 그대여 평생 말하고
Score: 0.0045, Sentence: 이제 사랑의 숨 쉬며 걸어요 함께 우리 내게는 가장 멋진 아버지 아빠 아니 그대 이제 조금은 쉬며 걸어요 아버지 아빠
Score: 0.0045, Sentence: 더 주지 못해 아쉬워 하는 당신께 더 무엇을 드려야 할지 엄마 나의 어머니 왜 이렇게 눈물이 나죠 가장 소중한 누구보다
Score: 0.0045, Sentence: 엄마 엄마 꿈이 그리워 그때는 모두다 그렇게 살았다고 텅빈 가슴 속 한편에다 묻어두고 엄마 엄마 그 시절이 그리워 엄마 엄마
Score: 0.0043, Sentence: 나의 아버지
Score: 0.0043, Sentence: 것 같아 세상에 하나뿐인 딸 나의 전부 나의 사랑 너만 있으면 살 수 있어 우리 예쁜 딸 세상에 가장 소중한


Category: 가족과의 관계
Top 10 Important Sentences with Scores:
Score: 0.0199, Sentence: 엄마 내가 너무 사랑한 사람

In [ ]:
import itertools
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# 문장의 단어 토큰화 및 그래프 구축 함수
def build_word_graph(sentence, window_size=2):
    words = word_tokenize(sentence.lower())  # 소문자 변환 및 토큰화
    G = nx.Graph()
    for i in range(len(words)):
        for j in range(i+1, min(i+window_size, len(words))):
            if words[i] != words[j]:
                G.add_edge(words[i], words[j])
    return G

# 카테고리별 문장에서 키워드 추출
category_keywords = defaultdict(list)

for category, sentences in final_ranked_sentences.items():
    for score, sentence in sentences:
        G = build_word_graph(sentence)
        scores = nx.pagerank(G)
        ranked_words = sorted(scores, key=scores.get, reverse=True)[:5]  # 상위 5개 키워드
        category_keywords[category].append(ranked_words)

# 카테고리별 주요 키워드 출력
for category, keywords in category_keywords.items():
    print(f"Category: {category}")
    print("Top Keywords:")
    for keyword_set in keywords:
        print(", ".join(keyword_set))
    print("\n")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Category: 가족
Top Keywords:
엄마, 사랑해요, 가장, 큰, 선물
엄마, 큰, 가장, 내게는, 선물
나의, 당신께, 더, 무엇을, 드려야
나의, 어머니, 당신은, 나죠, 가장
나의, 평생, 사랑, 그대여, 너만
이제, 아버지, 걸어요, 쉬며, 내게는
더, 소중한, 가장, 나죠, 눈물이
엄마, 그리워, 텅빈, 살았다고, 가슴
나의, 아버지
세상에, 딸, 나의, 같아, 가장


Category: 가족과의 관계
Top Keywords:
내가, 사랑한, 너무, 엄마, 사람
엄마, 아리랑
엄마, 사랑은, 아리랑, 우리
내가, 없는, 해준, 게, 엄마

나를, 사랑한, 너무, 엄마, 사람
나의, 엄마, 어머니
어머님, 그때마다, 말씀이
철이, 아들이, 없는, 나는, 되어서
이렇게, 좋은, 날에


Category: 감정
Top Keywords:
지금, 다시
우울해, 우울하다, 지금
참, 그리워, 네가, 다시
더, 그리워, 네가, 다시


나를, 혼자, 제발, 두지마
girl, ta, you, got, yeah
두, summer, 눈이, 비추던, 나를
let, a, 's, for, go


Category: 개인
Top Keywords:
내버려둬, 아름다워, oh
너만, 바라볼거야, 오직, oh
나, 떠나는, 그곳으로, 너와, 거야
어깨, hip, 무릎, 다, 머리
어깨, 다, 무릎, 머리, hip
어깨, 무릎, 머리, hip
나, 너와, 단둘이
걷고, 너와, 싶어져서
이런, 혹시나, 맘이
나랑, 같이, 혹시, 걸을래


Category: 꿈
Top Keywords:
한, 나를, 번, 다시, 믿고
(, 바람, 시원한, 이, oh


oh, babe, (, )
영원히, oh
oh, babe, (, )
high, chance, to, a, fly
high, 바람에, 높이, go, 올라타
goodbye, oh


Category: 사랑
Top Keywords:
내가, 더, 그댈, 오늘은, 많이
내게, 내가, 자꾸
조금씩, 너에게, 다가가, 날, 고백할
you,

In [ ]:
# 의미가 부족한 단어 리스트
meaningless_words = ['담은', '표현', '대한']

# 특수 문자를 제거하고 중복된 문장을 제거하는 함수
def remove_duplicates(sentences):
    cleaned_sentences = {}
    for sentence in sentences:
        cleaned_sentence = re.sub(r'[^\w\s]', '', sentence).lower()
        if cleaned_sentence not in cleaned_sentences:
            cleaned_sentences[cleaned_sentence] = sentence
    return list(cleaned_sentences.values())

# 긴 문장 분할
def split_long_sentences(sentences, max_length=20):
    split_sentences = []
    for sentence in sentences:
        words = sentence.split()
        split_sentences.extend([' '.join(words[i:i+max_length]) for i in range(0, len(words), max_length)])
    return split_sentences

# CSV 파일 로드
df = pd.read_csv('combined_dataset6.csv')

# 분류별 주요 문장을 저장할 딕셔너리
category_sentences = defaultdict(list)

# TextRank 알고리즘 적용
def apply_textrank(text, top_n=10):
    sentences = text.strip().split('\n')
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    cosine_sim_matrix = cosine_similarity(X, X)
    nx_graph = nx.from_numpy_array(cosine_sim_matrix)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    return ranked_sentences[:top_n]

# 각 카테고리별 중요한 문장 추출 및 키워드 선별
final_keywords = {}
for _, row in df.iterrows():
    category = row['소분류']
    lyrics = row['가사']
    important_sentences = apply_textrank(lyrics)
    category_sentences[category].extend([s for _, s in important_sentences])

# 중요한 단어 추출 및 의미가 부족한 단어 제거
for category, sentences in category_sentences.items():
    combined_text = " ".join(remove_duplicates(split_long_sentences(sentences)))
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([combined_text])
    feature_array = np.array(vectorizer.get_feature_names_out())
    tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]
    top_keywords = feature_array[tfidf_sorting][:20]  # 상위 20개 키워드 추출
    meaningful_keywords = [word for word in top_keywords if word not in meaningless_words]
    final_keywords[category] = meaningful_keywords[:10]  # 상위 10개 의미있는 키워드 선택

# 결과 출력
for category, keywords in final_keywords.items():
    print(f"Category: {category}")
    print("Top 10 Meaningful Keywords:", keywords)
    print("\n")

Category: 가족
Top 10 Meaningful Keywords: ['엄마', '내가', '나의', '사람', '이제', '아빠', '우리', '울어', '나는', '아버지']


Category: 가족과의 관계
Top 10 Meaningful Keywords: ['엄마', '내가', '아리랑', '너무', '그랬는지', '사람', '이제', '이렇게', '말씀이', '우리']


Category: 감정
Top 10 Meaningful Keywords: ['다시', '혼자', '네가', '우울하다', '그리워', '우울해', '니가', '챔피언', '나를', 'you']


Category: 개인
Top 10 Meaningful Keywords: ['oh', 'hip', '너와', '머리', '무릎', '더는', '않아', '어깨', '거야', '오직']


Category: 꿈
Top 10 Meaningful Keywords: ['oh', '날아올라', '너를', 'you', '나를', '길을', 'fly', 'high', '높이', 'go']


Category: 사랑
Top 10 Meaningful Keywords: ['내가', 'you', '너의', '너무', '그대', '많이', '너를', '지금', 'my', '좋은']


Category: 사랑의 다양한 양상
Top 10 Meaningful Keywords: ['사랑', '나의', '다시', '사람', '나를', '이젠', '내가', '너무', '그대', '번째']


Category: 사회 비판
Top 10 Meaningful Keywords: ['혼자', '나는', '사랑을', '함께', '있어', '나의', '언젠가', '그게', '못하는', '길에']


Category: 성장
Top 10 Meaningful Keywords: ['you', '거야', '내가', 'me', '나의', 'walk', '너의', '나를', 'with', '함께']


Category: 우정
Top 10 M

# 파일에 대해 적용_요약ver

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [ ]:
# CSV 파일 로드
df = pd.read_csv('combined_dataset6.csv')
df

,노래제목,가수,가사,요약,소분류,대분류
0,할아버지,여유와 설빈,할아버지는 살아계실 때 지금 내가 앉은 소파에 앉아 신호가 잘 잡히지 않는 라디오를...,"할아버지와의 추억과 그분의 삶을 회상하며, 할아버지에 대한 깊은 그리움과 사랑을 표...",가족,관계
1,아빠가 딸에게 (To.Jia),변진섭,햇살을 닮아 맘이 따뜻한 너에게 무슨 말을 할까 구름을 닮아 선한 눈빛을 가진 너에...,"부모의 따뜻한 마음과 바람을 담은 노래, 자녀에게 전하는 사랑과 삶의 교훈을 표현하...",가족,관계
2,딸에게 보내는 노래 (Vocal 성시경),토이,세상 모두 멈춘 것 같은 밤 방 안 가득 별빛 쏟아져 내려 지친 하루 피곤한 모습의...,어린 자녀와의 소중한 순간들을 회상하며 부모의 사랑과 희망을 담은 노래.,가족,관계
3,아빠와 함께 왈츠를 (Duet With 홍수연),김현철,아빠 나도 누군가를 사랑하게 될까 이 다음에 언젠가 내가 그런 때가 되면 아빤 내게...,아버지와 딸의 깊은 유대감과 미래의 사랑에 대한 소망을 표현한 노래,가족,관계
4,엄마와 딸,인순이,예쁘게 머리 땋아줘서 있는 짜증 없는 짜증 받아줘서 고마워 고마워 고마워 엄마 안기...,"엄마와 딸 사이의 깊은 사랑과 미안함, 그리고 서로에 대한 감사의 마음을 표현한 노래.",가족,관계
...,...,...,...,...,...,...
513,나에게로 떠나는 여행,NaN,저 푸른바다 끝까지\n말을 달리면\n소금같은 별이 떠있고\n사막엔 낙타만이 가는길\...,자유롭게 떠나고자 하는 강한 의지를 담은 노래,환경,일상
514,제주도의 푸른 밤,태연,떠나요 둘이서\n모든 걸 훌훌 버리고\n제주도 푸른 밤 그 별 아래\n이제는 더이상...,도시의 번잡함과 지쳐한 일상에서 벗어나 제주도의 평화로운 푸른 밤에 둘이서 행복한 ...,환경,일상
515,새들처럼,ft아일랜드,열린 공간 속을 가르며\n달려가는 자동차와\n석양에 비추인 사람들\nUh 도로 위를...,도심의 분주함에서 벗어나 파란 하늘 아래로 날아가고 싶어하는 자유로운 마음을 담은 노래,환경,일상
516,음파음파,레드벨벳,Umpah umpah umpah umpah\nUmpah umpah umpah ump...,해변에서의 여름 휴가와 자연을 즐기며 미소지음,환경,일상


In [ ]:
def apply_textrank(text, top_n=10):
    # 문장 분리
    sentences = text.strip().split('\n')
    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    # 코사인 유사도 행렬 계산
    cosine_sim_matrix = cosine_similarity(X, X)
    # 그래프 생성 및 PageRank 적용
    nx_graph = nx.from_numpy_array(cosine_sim_matrix)
    scores = nx.pagerank(nx_graph)
    # 점수가 높은 문장 추출
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    return ranked_sentences[:top_n]  # 상위 n개 문장과 점수 반환

import re

# 특수 문자를 제거하고 중복된 문장을 제거하는 함수
def remove_duplicates(sentences):
    cleaned_sentences = {}
    for sentence in sentences:
        # 특수 문자 제거 및 소문자 변환
        cleaned_sentence = re.sub(r'[^\w\s]', '', sentence).lower()
        if cleaned_sentence not in cleaned_sentences:
            cleaned_sentences[cleaned_sentence] = sentence
    return list(cleaned_sentences.values())

# 긴 문장 분할
def split_long_sentences(sentences, max_length=20):
    split_sentences = []
    for sentence in sentences:
        words = sentence.split()
        # 문장을 최대 길이로 나누어 리스트에 추가
        split_sentences.extend([' '.join(words[i:i+max_length]) for i in range(0, len(words), max_length)])
    return split_sentences

In [ ]:
# 분류별 주요 문장을 저장할 딕셔너리
category_sentences = defaultdict(list)

for _, row in df.iterrows():
    category = row['소분류']
    lyrics = row['요약']

    # TextRank 적용
    important_sentences = apply_textrank(lyrics)

    # 결과 저장
    category_sentences[category].extend([s for _, s in important_sentences])

In [ ]:
# 각 카테고리별 중요한 문장들에 대해 TextRank 재적용
final_ranked_sentences = {}
for category, sentences in category_sentences.items():
    combined_text = "\n".join(remove_duplicates(split_long_sentences(sentences)))
    final_ranked_sentences[category] = apply_textrank(combined_text, top_n=10)

In [ ]:
# 상위 10개 문장과 중요도 점수 출력
for category, sentences_scores in final_ranked_sentences.items():
    print(f"Category: {category}")
    print("Top 10 Important Sentences with Scores:")
    for score, sentence in sentences_scores:
        print(f"Score: {score:.4f}, Sentence: {sentence}")
    print("\n")

Category: 가족
Top 10 Important Sentences with Scores:
Score: 0.0268, Sentence: 어머니에 대한 감사와 사랑을 담은 노래.
Score: 0.0237, Sentence: 엄마와 딸 사이의 깊은 사랑과 미안함, 그리고 서로에 대한 감사의 마음을 표현한 노래.
Score: 0.0237, Sentence: 아버지의 희생과 사랑을 이해하고 그 마음에 대한 깊은 감사와 애정을 표현하는 노래.
Score: 0.0233, Sentence: 가족에 대한 깊은 사랑과 감사의 마음을 깨닫고, 그들에게 더 많은 사랑을 주겠다는 다짐을 표현하는 노래.
Score: 0.0233, Sentence: 어려웠던 아버지와의 관계를 회상하며, 성장한 후 아버지의 희생과 사랑을 이해하고 그에 대한 감사를 표현하는 노래.
Score: 0.0231, Sentence: 엄마에 대한 그리움과 사랑을 깊이 느끼며, 그녀의 부재 속에서 느끼는 슬픔과 외로움을 표현하는 노래.
Score: 0.0230, Sentence: 자신의 어린 시절과 부모님에 대한 사랑과 감사, 그리고 성장과 독립에 대한 이야기를 담은 노래.
Score: 0.0230, Sentence: 엄마의 끝없는 사랑과 배려, 그리고 그 사랑에 대한 딸의 감사와 미안함을 표현하는 노래.
Score: 0.0215, Sentence: 아픈 엄마에 대한 사랑과 미안함, 그리고 헤어짐을 앞둔 슬픔과 감사의 마음을 담은 노래.
Score: 0.0213, Sentence: 엄마에 대한 감사, 사랑, 미안함, 그리고 엄마를 향한 감정을 담은 노래.


Category: 가족과의 관계
Top 10 Important Sentences with Scores:
Score: 0.0979, Sentence: 어머니에 대한 애정과 감사의 마음을 노래
Score: 0.0920, Sentence: 아버지에 대한 사랑과 존경을 담은 감동적인 노래
Score: 0.0909, Sentence: 어머니에 대한 그리움과 사랑을 

In [ ]:
import itertools
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# 문장의 단어 토큰화 및 그래프 구축 함수
def build_word_graph(sentence, window_size=2):
    words = word_tokenize(sentence.lower())  # 소문자 변환 및 토큰화
    G = nx.Graph()
    for i in range(len(words)):
        for j in range(i+1, min(i+window_size, len(words))):
            if words[i] != words[j]:
                G.add_edge(words[i], words[j])
    return G

# 카테고리별 문장에서 키워드 추출
category_keywords = defaultdict(list)

for category, sentences in final_ranked_sentences.items():
    for score, sentence in sentences:
        G = build_word_graph(sentence)
        scores = nx.pagerank(G)
        ranked_words = sorted(scores, key=scores.get, reverse=True)[:5]  # 상위 5개 키워드
        category_keywords[category].append(ranked_words)

# 카테고리별 주요 키워드 출력
for category, keywords in category_keywords.items():
    print(f"Category: {category}")
    print("Top Keywords:")
    for keyword_set in keywords:
        print(", ".join(keyword_set))
    print("\n")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Category: 가족
Top Keywords:
대한, 노래, 감사와, 담은, 사랑을
딸, 노래, 사이의, 표현한, 깊은
희생과, 노래, 사랑을, 표현하는, 이해하고
대한, 노래, 깊은, 표현하는, 사랑과
아버지와의, 노래, 관계를, 표현하는, 회상하며
대한, 노래, 그리움과, 표현하는, 사랑을
대한, 어린, 노래, 시절과, 담은
끝없는, 노래, 사랑과, 표현하는, 배려
엄마에, 노래, 대한, 담은, 사랑과
,, 노래, 대한, 담은, 감사


Category: 가족과의 관계
Top Keywords:
대한, 마음을, 애정과, 감사의, 어머니에
대한, 감동적인, 사랑과, 담은, 존경을
대한, 담은, 그리움과, 사랑을, 어머니에
대한, 표현한, 깊은, 감사를, 사랑과
깊은, 담은, 유대감과, 마음을, 존경의
대한, 담아, 깊은, 아리랑에, 사랑과
대한, 따뜻하게, 애정과, 감정을, 감사의
추억과, 담은, 소통의, 중요성을, 아버지와의
함께하는, 가치를, 소중한, 시간의, 가족과
대한, 묘사한, 사랑과, 감성적으로, 존경을


Category: 감정
Top Keywords:
것에, 담은, 시달리며, 감정을, 비교하고
상실, 노래, ,, 담은, 그리고
감정을, 추억을, 다루며, 과거의, 고독한
피로와, 담은, 지침에, 메시지를, 대한
이에게, edm, 돌아가는, 표현한, 길에
속에서, 담은, 헤매고, 마음을, 불안해하는
,, 하는, 무거워질, 이해하고자, 쉽게
변화와, 담은, 감정의, 고뇌를, 파도
여행자의, 전하는, 마음을, 감정을, 담아
고독한, 이야기를, 마음을, 소녀의, 가지고


Category: 개인
Top Keywords:
사랑하는, 넘치는, 여성의, 자신감, 자신을
스타일과, 여성의, 자신감, 힙한, 넘치는
못하는, 댄스, 해변에서의, 담은, 추억과
이에게, 사랑의, 전하는, 감사와, 사랑하는
맞춘, 설레는, 그녀에게, 싶은, 마음을
열고, 감성적인, 진심으로, 고백하는, 마음을
우아하게, 강조하는, 춤추며, 스타일을, 자기만의
나이에, 담은, 도시에서,